# Tutorial: how to process lipidomics LC MS/MS data from PEAKS output file and MGF files

PEAKS is a proprietary software which, among many other things, is able to preprocess raw mass spec data, identify peaks and estimate their quality, align m/z values across multiple experiments and export a table of features as csv and MS2 spectra in MGF files. Currently this format is the most convenient way to input your data for `lipyd` until the MzML input module is under development. Here in the overview part I present the entire workflow and below I go through each step and show the customization options.

## Overview

In [3]:
import os
from lipyd import sample

basedir = ('/', 'home', 'denes' , 'archive', 'cihan')
exppath = ('gltpd1_intracellular_20180201', 'gltpd1_pos')
mgfpath = ('data', 'mgf')

fractions = (
    ('A9', ),
    ('A11', 'A12', 'B1')
)

peaksfile = os.path.join(*(basedir + exppath + ('feature.csv',)))
mgfdir = os.path.join(*(basedir + exppath + mgfpath))

reader = sample.SampleReader(
    input_type = 'peaks',
    fname = peaksfile,
)

samples = reader.get_sampleset(
    sampleset_args = {
        'ms2_param': {
            'mgfdir': mgfdir,
        },
        'ms2_format': 'mgf',
    }
)

samples.basic_filters()
samples.peak_size_filter(*fractions)
samples.database_lookup()
samples.ms2_identify()
samples.export_table(
    fname = '%sresults.tsv' % peaksfile,
    variables = ['peaksize'],
)

        Indexing SwissLipids -- finished: 100%|██████████| 623M/623M [00:19<00:00, 9.57Mit/s]


	:: Indexed 42556 records from `cache/LMSDFDownload12Dec17FinalAll.sdf`.


        Generating metabolites -- finished: 100%|██████████| 44.0/44.0 [00:15<00:00, 2.73it/s]
        Generating metabolites -- finished: 100%|██████████| 18.0/18.0 [00:07<00:00, 2.67it/s]
        Generating metabolites -- finished: 100%|██████████| 109/109 [00:14<00:00, 4.34it/s] 
        Generating metabolites -- finished: 100%|██████████| 1.00/1.00 [00:00<00:00, 7.19it/s]
        Generating metabolites -- finished: 100%|██████████| 1.00/1.00 [00:00<00:00, 94.3it/s]
        Analysing MS2 spectra -- finished: 100%|██████████| 392/392 [00:35<00:00, 19.4it/s]


Ok, it took only few seconds, or with loading the molecule databases maybe 2 minutes. Now let's see step by step with detailed comments.

# Loading modules and defining paths

We load only the modules `os` (to deal with paths) and `lipyd.sample`, I will tell more about the latter below.

In [4]:
import os
from lipyd import sample

First we need to define the location of our input files. The `basedir` contains all experiments. The `expparh` contains this particular experiment, I made it a separate tuple as you maybe have more experiments. The `mgfpath` is the subdirectory for MGF files, PEAKS just outputs these usually in this directory.

In [5]:
basedir = ('/', 'home', 'denes' , 'archive', 'cihan')
exppath = ('gltpd1_intracellular_20180201', 'gltpd1_pos')
mgfpath = ('data', 'mgf')

Also if you have more than one experiments you can use `os.listdir` to find all subdirectories in `basedir`:

In [17]:
skip = {'blanks and standards', 'results.tar.gz'}
experiments = [
    os.path.join(*(basedir + (dir0, dir1)))
    for dir0 in os.listdir(os.path.join(*basedir))
    for dir1 in os.listdir(os.path.join(*(basedir + (dir0,))))
    if dir0 not in skip
]

In this case we got a list of directories each containing one experiment:

In [19]:
experiments

['/home/denes/archive/cihan/bpifb2_intracellular_20180201/bpifb2_intracellular_20180201_neg',
 '/home/denes/archive/cihan/bpifb2_intracellular_20180201/bpifb2_intracellular_20180201_pos',
 '/home/denes/archive/cihan/bpifb2_secreted_20180118/bpifb2_secreted_20180118_pos',
 '/home/denes/archive/cihan/bpifb2_secreted_20180118/bpifb2_secreted_20180118_neg',
 '/home/denes/archive/cihan/bpifb2_intracellular_20180124/bpifb2_intracellular_20180124_neg',
 '/home/denes/archive/cihan/bpifb2_intracellular_20180124/bpifb2_intracellular_20180124_pos',
 '/home/denes/archive/cihan/gltpd1_intracellular_20180201/gltpd1_pos',
 '/home/denes/archive/cihan/gltpd1_intracellular_20180201/gltpd1_neg']

Anyways let's analyse now only the GLTPD1 positive. The PEAKS output file normally called `feature.csv`.

In [20]:
peaksfile = os.path.join(experiments[6], 'feature.csv')

And the directory with MGF files:

In [23]:
mgfdir = os.path.join(*((experiments[6],) + mgfpath))

Now we have the locations of the input files defined. Continue with reading the data.

# Reading data

You may know PEAKS names columns like `GLTPD1_pos_A11_m/z`, `GLTPD1_pos_A11_Normalized Area` and so on. These names come from your settings and you can provide a custom method to extract information from them e.g. the protein name, the SEC fraction, the ion mode. Here we define a method which takes the column label and returns the attributes as a dictionary.

In [27]:
import re

def label_processor(label):
    
    relabel = re.compile(r'(.*)_([A-Z])([0-9]{1,2})_(neg|pos)')
    
    match = relabel.search(label)

    if match:

        main, row, col, ionmode = match.groups()
        col = int(col)

        return {
            'main': main,
            'fraction': (row, col),
            'ionmode': ionmode,
        }

We can test our method:

In [28]:
label_processor('GLTPD1_A11_pos')

{'main': 'GLTPD1', 'fraction': ('A', 11), 'ionmode': 'pos'}

Ok this looks all right. Also you may want to order your samples some logical way, e.g. as the SEC fractions come like A6, A7, A8, etc. For this we can define a custom method which sorts the labels. The labels are under the `label` key in the dictionary of sample attributes and if we want to sort by fractions just do like this:

In [29]:
def sample_sorter(samples):
        
    return sorted(samples, key = lambda s: s['label']['fraction'])

Now we can read the PEAKS file with providing the methods above to make sure our sample attributes and order will be set correctly. For this we will use the `lipyd.sample.SampleReader` class. This class calls the appropriate reader, reads the data and creates objects which will later perform the analysis. At the moment the only available reader is `PeaksReader`, but soon you can call the MzML reader the same way. All parameters for the reader we can pass to `SampleReader` directly and it will forward them to the actual reader. For example our methods `label_processor` and `sample_sorter` will be forwarded to `lipyd.reader.PeaksReader`.

In [30]:
reader = sample.SampleReader(
    input_type = 'peaks',
    fname = peaksfile,
    label_processor = label_processor,
    sample_sorter = sample_sorter,
)

We got this object:

In [31]:
reader

It has already the data read from the PEAKS file. For example we can find the m/z values here (I just print the first 3 rows):

In [34]:
reader.reader.mzs[:3]

array([[722.5222, 722.5223, 722.522 , 722.5222, 722.5225, 722.5223],
       [682.3596, 682.3588, 682.3594, 682.3594, 682.3596, 682.359 ],
       [261.137 , 261.137 , 261.137 , 261.137 , 261.1369, 261.1377]])

And the sample attributes processed by our `label_processor` method (good to check before proceeding):

In [35]:
reader.reader.samples

[{'label_raw': 'gltpd1_A9_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 9), 'ionmode': 'pos'},
  'm/z': 7,
  'RT mean': 8,
  'Normalized Area': 9},
 {'label_raw': 'gltpd1_A10_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 10), 'ionmode': 'pos'},
  'm/z': 10,
  'RT mean': 11,
  'Normalized Area': 12},
 {'label_raw': 'gltpd1_A11_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 11), 'ionmode': 'pos'},
  'm/z': 16,
  'RT mean': 17,
  'Normalized Area': 18},
 {'label_raw': 'gltpd1_A12_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 12), 'ionmode': 'pos'},
  'm/z': 19,
  'RT mean': 20,
  'Normalized Area': 21},
 {'label_raw': 'gltpd1_B1_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('B', 1), 'ionmode': 'pos'},
  'm/z': 22,
  'RT mean': 23,
  'Normalized Area': 24},
 {'label_raw': 'gltpd1_B2_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('B', 2), 'ionmode': 'pos'},
  'm/z': 13,
  'RT mean': 14,
  'Normalized Area': 15}]

And essential to check if the reader could find out the ion mode, if it could not we can set it explicitely later:

In [37]:
reader.reader.ionmode

'pos'

The reading went all right, we can retrieve a `SampelSet` object:

In [63]:
samples = reader.get_sampleset()

# Working with the SampleSet object

As a next step you may want to apply some trivial filters which usually discard like 90% of the features. Now we have the `SampleSet` object which represents a series of LC MS/MS runs. In the `lipyd.sample` module you can find also the `Sample` class which represents one single run. But these work mostly a similar way. In summary, these objects can harbor an arbitrary number of data arrays (variables). Only restriction is that they must be the same length (obviously as all arrays contain data about the same series of features). Let's see which data this `SampleSet` object contains:

In [39]:
samples.var

{'intensities', 'mzs', 'mzs_by_sample', 'rts'}

See it has m/z's, intensities, retention times. These are all arrays, for example printing only 3 rows:

In [41]:
samples.intensities[:3]

array([[37600000., 38400000., 41200000., 38500000., 37700000., 35500000.],
       [13400000., 15300000., 16700000., 16600000., 16600000., 15100000.],
       [31600000., 31100000., 29100000., 27400000., 29200000.,  8550000.]])

The `SampleSet` object has a `FeatureAttrbutes` object with all the data which belong directly to features (`SampleSet` is for data related to samples vs. features).

In [42]:
samples.feattrs

Here we find for example the quality metrics calculated by PEAKS:

In [43]:
samples.feattrs.quality[:10]

array([3.24, 2.55, 2.51, 2.45, 2.17, 2.14, 2.13, 2.06, 2.01, 2.  ])

Importantly these arrays all sort together if you call `sort_all` on any of the objects. Same stands for `filter`. If I sort the feature attributes by their centroid m/z, all other arrays in the sample set will be sorted the same way:

In [47]:
samples.feattrs.sort_all('centr_mzs')

In [50]:
samples.mzs_by_sample[:5]

array([[     nan,      nan, 250.2248,      nan,      nan,      nan],
       [     nan, 251.1338, 251.1345, 251.1339,      nan,      nan],
       [251.1499, 251.1497, 251.1496, 251.1498, 251.1501, 251.1496],
       [251.1942, 251.1941, 251.1942, 251.1941, 251.1941, 251.1942],
       [252.1317,      nan,      nan,      nan,      nan,      nan]])

You see now it starts from lowest m/z (around 250) and goes ascending order. Also you see whereever a feature is missing from a sample we have `numpy.nan` which means missing data.

And how do we know which sample is which one? The columns in the arrays correspond to samples. In the `attrs` attribute we have the sample attributes exactly how we created them by our custom methods above:

In [53]:
samples.attrs

[{'label_raw': 'gltpd1_A9_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 9), 'ionmode': 'pos'},
  'm/z': 7,
  'RT mean': 8,
  'Normalized Area': 9},
 {'label_raw': 'gltpd1_A10_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 10), 'ionmode': 'pos'},
  'm/z': 10,
  'RT mean': 11,
  'Normalized Area': 12},
 {'label_raw': 'gltpd1_A11_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 11), 'ionmode': 'pos'},
  'm/z': 16,
  'RT mean': 17,
  'Normalized Area': 18},
 {'label_raw': 'gltpd1_A12_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('A', 12), 'ionmode': 'pos'},
  'm/z': 19,
  'RT mean': 20,
  'Normalized Area': 21},
 {'label_raw': 'gltpd1_B1_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('B', 1), 'ionmode': 'pos'},
  'm/z': 22,
  'RT mean': 23,
  'Normalized Area': 24},
 {'label_raw': 'gltpd1_B2_pos ',
  'label': {'main': 'gltpd1', 'fraction': ('B', 2), 'ionmode': 'pos'},
  'm/z': 13,
  'RT mean': 14,
  'Normalized Area': 15}]

Now go forward for filtering the features.

# Filtering

You can easily filter the features by any of the variables. Let's say you want to remove features with qulity below 0.2.

In [55]:
import numpy
lowquality = numpy.where(samples.feattrs.quality < 0.2)

We got an array with indices of the features with quality below 0.2:

In [57]:
lowquality

(array([   0,    4,    6, ..., 8474, 8481, 8482]),)

Before filtering check the number of features:

In [58]:
len(samples)

8484

We apply a negative filter i.e. these features should be removed:

In [65]:
samples.filter(lowquality, negative = True)

Many features have been removed:

In [67]:
len(samples)

4050

But this was only an example how you can make your own filter. The `SampleSet` object has these trivial filters build in, you can simply do:

In [68]:
samples.basic_filters()

And see how many samples remained:

In [69]:
len(samples)

1209

# Filtering by intensity profiles

1,209 is still quite some features and probably you want to keep only the ones enriched in your experimental subject, for example here the samples containing GLTPD1. To do this we apply a simple filter which selects the features with at least 2 times higher intensity in any of the protein containing samples compared to the background samples. Of course you can apply some more sophisticated filter.

First define the background and the protein containing samples. Fraction A9 is background while A11, A12 and B1 contain GLTPD1.

In [70]:
fractions = (
    ('A9', ),
    ('A11', 'A12', 'B1')
)